In [15]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import pandas as pd

In [16]:
dataset = pd.read_json("../Datasets/Sarcasm_Headlines_Dataset.json", lines=True)
dataset.head()

# shuffled_df = dataset.sample(frac=1).reset_index(drop=True)

half_df = dataset.head(len(dataset)//2)
dataset = half_df

In [17]:
column_name_to_remove = 'article_link'
dataset = dataset.drop(columns=[column_name_to_remove])

dataset.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [18]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences


X_train, X_test, y_train, y_test = train_test_split(dataset['headline'], dataset['is_sarcastic'], test_size=0.2, random_state=42)

# Tokenize and vectorize the training text data using Tokenizer and pad_sequences
max_length = 100
tokenizer = Tokenizer()   #lower=False
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=max_length)

# Tokenize and vectorize the testing text data using the same Tokenizer
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=max_length)

In [19]:
from keras.layers import CuDNNLSTM
from keras.models import Sequential
from keras.layers import Embedding, Dense
from keras.optimizers import Adam

embedding_dim = 100

# Define the vocabulary size based on the actual number of unique words in the training data
vocab_size = len(tokenizer.word_index) + 1

max_length = 100
optimizer = Adam(learning_rate=0.00001)
m1 = Sequential()
m1.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
m1.add(CuDNNLSTM(units=150))
m1.add(Dense(units=64))
m1.add(Dense(units=64))
m1.add(Dense(units=1, activation='sigmoid'))

m1.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
m1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 100)          1865900   
                                                                 
 cu_dnnlstm_2 (CuDNNLSTM)    (None, 150)               151200    
                                                                 
 dense_6 (Dense)             (None, 64)                9664      
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,030,989
Trainable params: 2,030,989
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Train the model
m1.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = m1.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy * 100:.2f}%')

Epoch 1/20
167/167 [==============================] - 3s 15ms/step - loss: 0.6911 - accuracy: 0.5471 - val_loss: 0.6900 - val_accuracy: 0.5425
Epoch 2/20
167/167 [==============================] - 2s 14ms/step - loss: 0.6867 - accuracy: 0.5565 - val_loss: 0.6884 - val_accuracy: 0.5425
Epoch 3/20
167/167 [==============================] - 2s 14ms/step - loss: 0.6840 - accuracy: 0.5565 - val_loss: 0.6875 - val_accuracy: 0.5425
Epoch 4/20
167/167 [==============================] - 2s 14ms/step - loss: 0.6817 - accuracy: 0.5565 - val_loss: 0.6852 - val_accuracy: 0.5425
Epoch 5/20
167/167 [==============================] - 2s 14ms/step - loss: 0.6782 - accuracy: 0.5565 - val_loss: 0.6814 - val_accuracy: 0.5425
Epoch 6/20
167/167 [==============================] - 2s 14ms/step - loss: 0.6715 - accuracy: 0.5565 - val_loss: 0.6727 - val_accuracy: 0.5425
Epoch 7/20
167/167 [==============================] - 2s 14ms/step - loss: 0.6561 - accuracy: 0.5604 - val_loss: 0.6543 - val_accuracy: 0.5492

In [21]:
from sklearn.metrics import precision_score, recall_score

# Predict on validation data
y_val_pred_prob_m1 = m1.predict(X_test)
y_val_pred_m1 = (y_val_pred_prob_m1 > 0.5).astype(int)  # Threshold for binary classification

# Assuming y_test is in binary format (0 or 1)
y_val_true_m1 = y_test

# Calculate precision and recall for binary classification
precision_m1 = precision_score(y_val_true_m1, y_val_pred_m1)
recall_m1 = recall_score(y_val_true_m1, y_val_pred_m1)

# print the results
print(f'Precision: {precision_m1:.4f}')
print(f'Recall: {recall_m1:.4f}')

84/84 [==============================] - 1s 7ms/step
Precision: 0.8060
Recall: 0.8126
